In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [13:23<00:00, 18.23it/s]


In [12]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [13]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [15]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [16]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [17]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [18]:
dict_genres = dict(cuenta_generos.most_common(24))

In [19]:
fuzz.ratio('hip-hop', "hip hop")

86

In [20]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [21]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [22]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
25847,Antidote,199,2017-04-12,Travis Scott,https://open.spotify.com/track/1wHZx0LgzFHyeIZ...,United States,top200,NEW_ENTRY,173982.000,"23 Aug 2022, 04:28","16 Jun 2013, 00:08","Jacques Webster (born April 30, 1992), better ...","Jacques Webster (born April 30, 1992), better ...",Hip-Hop,rap,trap,hip hop,american,trap,rap,Hip-Hop,psychedelic,travis scott,spotify:track:1wHZx0LgzFHyeIZkUydNXq,0.713,0.526,1.000,-5.046,1.000,0.032,0.008,0.000,0.124,0.131,131.050,audio_features,1wHZx0LgzFHyeIZkUydNXq,https://api.spotify.com/v1/tracks/1wHZx0LgzFHy...,https://api.spotify.com/v1/audio-analysis/1wHZ...,262693.000,4.000,151,79,hip-hop,hip-hop
213700,STOP TRYING TO BE GOD,22,2018-08-08,Travis Scott,https://open.spotify.com/track/1xYpVRspgaLxdJL...,United States,top200,SAME_POSITION,782324.000,"07 May 2022, 18:01","16 Jun 2013, 00:08","Jacques Webster (born April 30, 1992), better ...","Jacques Webster (born April 30, 1992), better ...",Hip-Hop,rap,trap,hip hop,american,Psychedelic Soul,psychedelic,trap,alternative rnb,Hip-Hop,spotify:track:1xYpVRspgaLxdJLOTQkTjg,0.588,0.567,4.000,-3.738,0.000,0.063,0.221,0.000,0.135,0.113,150.009,audio_features,1xYpVRspgaLxdJLOTQkTjg,https://api.spotify.com/v1/tracks/1xYpVRspgaLx...,https://api.spotify.com/v1/audio-analysis/1xYp...,338438.000,4.000,7,47,hip-hop,hip-hop
72643,Transportin',110,2018-01-17,Kodak Black,https://open.spotify.com/track/1WIZiOuNO3woKfd...,United States,top200,MOVE_UP,255915.000,NaN,"09 Apr 2016, 10:24","Bill Kahan Kapri (born as Dieuson Octave) , (b...","Bill Kahan Kapri (born as Dieuson Octave) , (b...",Hip-Hop,rap,trap,hip hop,florida,NaN,NaN,NaN,NaN,NaN,spotify:track:1WIZiOuNO3woKfdlSK2gNn,0.804,0.660,10.000,-4.860,0.000,0.073,0.032,0.000,0.103,0.257,94.053,audio_features,1WIZiOuNO3woKfdlSK2gNn,https://api.spotify.com/v1/tracks/1WIZiOuNO3wo...,https://api.spotify.com/v1/audio-analysis/1WIZ...,169345.000,4.000,30,212,hip-hop,hip-hop
423987,D Rose,183,2017-11-07,Lil Pump,https://open.spotify.com/track/1fEl5TPKRJAsuP6...,United States,top200,MOVE_UP,198788.000,"16 May 2021, 16:23","08 Jun 2017, 05:05","Gazzy Garcia (born August 17, 2000), known pro...","Gazzy Garcia (born August 17, 2000), known pro...",trap,Hip-Hop,rap,mumble rap,hip hop,Hip-Hop,rap,2010s,played,Lil pump,spotify:track:1fEl5TPKRJAsuP6TqZ23hB,0.781,0.553,0.000,-4.277,1.000,0.073,0.201,0.000,0.383,0.308,121.968,audio_features,1fEl5TPKRJAsuP6TqZ23hB,https://api.spotify.com/v1/tracks/1fEl5TPKRJAs...,https://api.spotify.com/v1/audio-analysis/1fEl...,135497.000,4.000,67,51,trap,trap
136625,Watermelon Sugar,5,2020-06-19,Harry Styles,https://open.spotify.com/track/6UelLqGlWMcVH1E...,United States,top200,MOVE_UP,901907.000,"11 Aug 2020, 01:00","24 Aug 2011, 17:11",Harry Edward Styles (born 1 February 1994 in R...,Harry Edward Styles (born 1 February 1994 in R...,pop,rock,pop rock,alternative,seen live,pop,rock,2019,indie pop,2010s,spotify:track:6UelLqGlWMcVH1E5c4H7lY,0.548,0.816,0.000,-4.209,1.000,0.046,0.122,0.000,0.335,0.557,95.390,audio_features,6UelLqGlWMcVH1E5c4H7lY,https://api.spotify.com/v1/tracks/6UelLqGlWMcV...,https://api.spotify.com/v1/audio-analysis/6Uel...,174000.000,4.000,3,755,pop,pop


In [23]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [24]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [25]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop


In [26]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [27]:
df['gender'].value_counts() / df.shape[0]

male         0.478
female       0.136
group        0.136
non-binary   0.016
Name: gender, dtype: float64

In [34]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
90744,20 Min,156,2020-08-18,Lil Uzi Vert,https://open.spotify.com/track/0uxSUdBrJy9Un0E...,United States,top200,SAME_POSITION,233062.000,"26 Mar 2021, 14:59","21 Apr 2016, 19:11","Symere Bysil Woods (born July 31, 1995), known...","Symere Bysil Woods (born July 31, 1995), known...",trap,Hip-Hop,rap,hip hop,emo rap,trap,Hip-Hop,rap,hip hop,experimental,spotify:track:0uxSUdBrJy9Un0EYoBowng,0.773,0.750,8.000,-4.009,0.000,0.117,0.109,0.000,0.174,0.783,123.426,audio_features,0uxSUdBrJy9Un0EYoBowng,https://api.spotify.com/v1/tracks/0uxSUdBrJy9U...,https://api.spotify.com/v1/audio-analysis/0uxS...,220587.000,4.000,53,665,trap,trap,NaN,"July 31, 1995"
297547,How I Move (feat. Lil Baby),191,2020-03-10,Flipp Dinero,https://open.spotify.com/track/14X1WxiHXdVRJ99...,United States,top200,MOVE_UP,245182.000,NaN,"27 Oct 2018, 16:25","Christopher Saint Victor (born November 16, 19...","Christopher Saint Victor (born November 16, 19...",Hip-Hop,rap,trap,cloud rap,United States,NaN,NaN,NaN,NaN,NaN,spotify:track:14X1WxiHXdVRJ99cMCBzOE,0.826,0.635,5.000,-5.181,0.000,0.212,0.102,0.000,0.245,0.808,119.954,audio_features,14X1WxiHXdVRJ99cMCBzOE,https://api.spotify.com/v1/tracks/14X1WxiHXdVR...,https://api.spotify.com/v1/audio-analysis/14X1...,224916.000,4.000,95,85,hip-hop,hip-hop,male,"November 16, 1995"
206086,Bigger Than You (feat. Drake & Quavo),87,2018-07-23,2 Chainz,https://open.spotify.com/track/5S1IUPueD0xE0vj...,United States,top200,MOVE_DOWN,302544.000,NaN,"03 Nov 2011, 08:26","Tauheed Epps (born September 12, 1976), better...","Tauheed Epps (born September 12, 1976), better...",Hip-Hop,rap,trap,hip hop,Dirty South,NaN,NaN,NaN,NaN,NaN,spotify:track:5S1IUPueD0xE0vj4zU3nSf,0.888,0.515,1.000,-6.246,1.000,0.271,0.247,0.000,0.446,0.264,141.009,audio_features,5S1IUPueD0xE0vj4zU3nSf,https://api.spotify.com/v1/tracks/5S1IUPueD0xE...,https://api.spotify.com/v1/audio-analysis/5S1I...,225893.000,4.000,2,116,hip-hop,hip-hop,male,"September 12, 1976"
322270,High Without Your Love,171,2017-07-07,Loote,https://open.spotify.com/track/7qs2GAglLWfxEfm...,United States,top200,MOVE_DOWN,187069.000,NaN,"20 Jun 2019, 23:35",Loote is an American pop duo based in New York...,Loote is an American pop duo based in New York...,pop,electronic,seen live,electropop,new york,NaN,NaN,NaN,NaN,NaN,spotify:track:3hwyRkhcVkcgUJHKKmxS8k,0.871,0.493,5.000,-5.865,1.000,0.074,0.123,0.000,0.132,0.769,102.011,audio_features,3hwyRkhcVkcgUJHKKmxS8k,https://api.spotify.com/v1/tracks/3hwyRkhcVkcg...,https://api.spotify.com/v1/audio-analysis/3hwy...,191227.000,4.000,10,46,pop,pop,non-binary,NaN
421328,The 1975,47,2020-05-29,The 1975,https://open.spotify.com/track/0YtmwGMUPaBIEuJ...,United States,viral50,MOVE_DOWN,NaN,"18 Sep 2016, 17:28","19 Mar 2012, 21:49",The 1975 are an English band formed in 2002 in...,The 1975 are an English band formed in 2002 in...,indie,indie rock,seen live,british,alternative,indie rock,ambient,10s,Greta Thunberg,pop,spotify:track:7omxuGYsym1n2RlVf86OF9,0.542,0.433,9.000,-10.470,1.000,0.030,0.897,0.173,0.101,0.495,140.939,audio_features,7omxuGYsym1n2RlVf86OF9,https://api.spotify.com/v1/tracks/7omxuGYsym1n...,https://api.spotify.com/v1/audio-analysis/7omx...,250900.000,4.000,27,8,indie,indie,group,NaN


Age from artist

-- taking the data from the summary

In [29]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w* \d*, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

December 2, 1998     10839
July 4, 1995         10700
January 23, 1998      9272
October 24, 1986      8719
December 18, 2001     7645
                     ...  
April 14, 1993           1
May 6, 2008              1
 1970, 1971              1
August 19, 1986          1
August 30, 1997          1
Name: birthday, Length: 999, dtype: int64

In [30]:
df.sample()

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
265395,Sixteen,91,2019-05-11,Ellie Goulding,https://open.spotify.com/track/4PkIDTPGedm0enz...,United States,top200,MOVE_UP,329275.000,"20 May 2019, 15:29","12 May 2008, 15:32","Ellie Goulding (born Elena Jane Goulding, 30 D...","Ellie Goulding (born Elena Jane Goulding, 30 D...",female vocalists,electronic,british,indie,indie pop,pop,electropop,dance,2010s,female vocalists,spotify:track:4PkIDTPGedm0enzdvilLNd,0.669,0.801,8.000,-5.445,1.000,0.136,0.268,0.000,0.059,0.541,109.919,audio_features,4PkIDTPGedm0enzdvilLNd,https://api.spotify.com/v1/tracks/4PkIDTPGedm0...,https://api.spotify.com/v1/audio-analysis/4PkI...,201072.000,4.000,50,55,female vocalists,female vocalists,female,NaN


# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df

Genre distibution

How was the music during covid